# Deep Hedging AI
### Vanilla Deep Hedging engine reference implementation with dynamic training update.

This is the main example notebook. It shows learning to hedge a vanilla call option (ATM by default), first in a Black & Scholes world with statistical drift, and secondly in a world where a second option can be traded.
The examples are not intended to be overly realistic.

In the Black & Scholes case we see that the hedge learned is _not_ the risk-neutral hedge. The notebook <tt>trainer-bs_nodrift.ipynb</tt> demonstrates that if the statistical drift is zero, and step size is sufficiently small (daily), then the Deep Hedging hedge approximates the risk-neutral hedge.

### Hans Buehler, June 30 2022


In [1]:
""" Slighly annoying: by default the SageMaker Python import directory does not include our git directory """
#!pip -q install cdxbasics "tensorflow>=2.10" "tensorflow_probability>=0.14"
import os
p = os.getcwd()
dhn = "/deephedging/"
i = p.find(dhn)
if i!=-1:
    p = p[:i]
    import sys
    if not p in sys.path:
        sys.path.append(p)
        print("Added python path %s" % p)


In [ ]:
print("Deep Hedging AI says hello ... ", end='')
from cdxbasics.config import Config
from deephedging.v4.trainer import train
from deephedging.v4.gym import VanillaDeepHedgingGym
from deephedging.v4.world import SimpleWorld_Spot_ATM

from IPython.display import display, Markdown

# see print of the config below for numerous options
config = Config()
# world
config.world.samples = 10000
config.world.steps = 20
config.world.black_scholes = True
# gym
config.gym.objective.utility = "exp2"
config.gym.objective.lmbda = 1.
config.gym.agent.depth = 3
config.gym.agent.activation = "softplus"

#config.gym.init_agent.features = []
#config.gym.objective.features = []
# trainer
config.trainer.train.optimizer.name = "adam"
config.trainer.train.batch_size = None
config.trainer.train.epochs = 400
config.trainer.caching.mode = "on"
if False:
    config.trainer.visual.epoch_refresh = 5
    config.trainer.visual.confidence_pcnt_lo = 0.25
    config.trainer.visual.confidence_pcnt_hi = 0.75

display(Markdown("## Deep Hedging in a Black \& Scholes World"))

# create world
world  = SimpleWorld_Spot_ATM( config.world )
val_world  = world.clone(samples=world.nSamples//10)

# create training environment
r = train( world=world, val_world=val_world, config=config )
print("Keys of the dictionary returned by the gym: ", r.keys())

print("=========================================")
print("Config usage report")
print("=========================================")
print( config.usage_report() )
config.done()

Deep Hedging AI says hello ... 

## Deep Hedging in a Black \& Scholes World

00: Initializing training for VanillaDeepHedging
01:   VanillaDeepHedging: model generated. Preparing training for 400 epochs. Model has 10369 trainable weights.
02:     'VanillaDeepHedging is using a total of 10369 weights: 6111 for the main agent, 1506 for 5 initial states and delta, and 2752 weights for the two exp2@1 monetary utilities
02:      Features available for the agent per time step:   action, cost, delta, ivol, lbnd_a, pnl, price, spot, sqrt_time_left, time_left and ubnd_a
02:      Features used by the agent per time step:         delta, price and time_left
02:      Features available for the initial agent:         cost, ivol, lbnd_a, price, spot, sqrt_time_left, time_left and ubnd_a
02:      Features used by the initial agent:               price
02:      Features available for the monetary utilities:    cost, ivol, lbnd_a, price, spot, sqrt_time_left, time_left and ubnd_a
02:      Features used by the monetary utilities:          price
02:     Number of time steps: 20
02

#### Comparison to Black & Scholes


In [ ]:
import importlib as imp
import packages.cdx_tf.cdx_tf.monetary_utility as monetary_utility
imp.reload(monetary_utility)
import deephedging.v4.plot_bs_hedge as plot_bs_hedge
imp.reload(plot_bs_hedge)

plot_bs_hedge.plot_blackscholes( world, r.gym, config )

## Stochastic Vol
### Trading with two assets

In [ ]:
print("Deep Hedging AI says hello  ... ", end='')
from cdxbasics.config import Config
from deephedging.trainer import train
from deephedging.gym import VanillaDeepHedgingGym
from deephedging.world import SimpleWorld_Spot_ATM
import tensorflow as tf

from IPython.display import display, Markdown

# see print of the config below for numerous options
config = Config()
# world
config.world.samples = 10000
config.world.steps = 20
config.world.black_scholes = False
# gym
config.gym.objective.utility = "cvar"
config.gym.objective.lmbda = 1.
config.gym.agent.network.depth = 3
config.gym.agent.network.activation = "softplus"
# trainer
config.trainer.train.optimizer.name = "adam"
config.trainer.train.optimizer.learning_rate = 0.001
config.trainer.train.optimizer.clipvalue = 1.
config.trainer.train.optimizer.global_clipnorm = 1.
config.trainer.train.batch_size = None
config.trainer.train.epochs = 800
config.trainer.caching.mode = "on"
config.trainer.train.run_eagerly = None
config.trainer.visual.epoch_refresh = 5
config.trainer.visual.confidence_pcnt_lo = 0.25
config.trainer.visual.confidence_pcnt_hi = 0.75

display(Markdown("## Deep Hedging in a simple Stochastic Volatility World"))

# create world
world  = SimpleWorld_Spot_ATM( config.world )
val_world  = world.clone(samples=world.nSamples//10)

# create training environment
tf.debugging.enable_check_numerics()
gym = VanillaDeepHedgingGym( config.gym )

# create training environment
train( gym=gym, world=world, val_world=val_world, config=config.trainer )
r = gym(world.tf_data)
print("Keys of the dictionary returned by the gym: ", r.keys())

print("=========================================")
print("Config usage report")
print("=========================================")
print( config.usage_report() )
config.done()